In [1]:
#安装transformers库
!pip install transformers accelerate scikit-learn numpy pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


Load the tokenizer:

In [2]:
import os

data_disk_dir = "autodl-tmp"  # 数据盘dir（云GPU平台）
model_name = "zh2ko_based_on_nllb-200-distilled-600M"
model_dir = os.path.join(data_disk_dir, model_name)

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_dir, tgt_lang=None)

Prepare your training and validation data:

In [4]:
data_dir = "data/027.한국어-중국어_번역_말뭉치_1/01.데이터"
csv_train = os.path.join(data_dir, "1.Training/원천데이터/ko2zh_patent_1_training.csv")
csv_valid = os.path.join(data_dir, "2.Validation/원천데이터/ko2zh_patent_2_validation.csv")

In [5]:
rename_dict = {"중국어": "zho_Hans", "한국어": "kor_Hang"}

In [6]:

import pandas as pd

#load Training Data
df_train = pd.read_csv(csv_train)[rename_dict.keys()]
df_train.rename(columns=rename_dict, inplace=True)  # rename the columns

#load Validation Data
df_valid = pd.read_csv(csv_valid)[rename_dict.keys()]
df_valid.rename(columns=rename_dict, inplace=True)  # rename the columns

In [7]:
df_train

,zho_Hans,kor_Hang
0,"本发明可以检测通过插头供应的主电源的频率特性,并为使从接近该频率零电位的上升电位连接到继电器...",본 발명은 플러그를 통해 공급되는 주전원의 주파수 특성을 검출하여 주파수의 제로전위...
1,"本发明是关于蜂窝电话的发明,当由于手机的移动而离开服务区域超过预定时间时自动关闭电源,并以预...","본 발명은 셀룰라폰에 관한 것으로, 셀룰라폰의 이동으로 소정시간이상 서어비스 지역을..."
2,"另外,因为不执行利用单元多功能复制的资源管理,所以存在无法向多个用户提供同一个音调的缺点。","또한, 셀 멀티 카피 기능을 활용하기 위한 자원 관리를 수행하지 않기 때문에 하나의..."
3,"安装有编码器的电动机,可以测量回转量和回转速度。",엔코더가 부착된 모터의 경우 회전량과 회전 속도를 측정할 수 있다.
4,"本发明是关于一种用于保护真空泵的电动机,免受半导体溅射(Sputtering)设备中的过电流...","본 발명은 반도체 스퍼터링(Sputtering)장비에 있어서, 진공 펌프의 모터를 ..."
...,...,...
119995,本发明涉及一种用于在提供背景声音呼叫服务时改善背景音乐的声音质量方法。,본 발명은 배경음 통화 서비스 제공시에 배경음악의 음질을 향상시키기 위한 방법에 관...
119996,"可以将电流释放阻抗设置得较低,提供一种高度方便的电池装置。","과전류 해제 임피던스를 낮게 설정할 수 있어, 편리성이 높은 배터리 장치를 제공한다."
119997,"本发明的目的是优化密封构件的布置,该密封构件阻止工艺腔体的内部与大气之间的连通。",본 발명의 과제는 처리실 내와 대기의 연통을 차단하는 밀봉 부재의 배치의 적정화를 ...
119998,"另外,本发明包括固定体缓冲装置,因此减轻冲击来获得防止绝缘体破裂的效果。","또한, 본 발명은 고정체 완충수단이 포함되어 있어 충격을 완화시킴으로써 애자 파손을..."


In [8]:
df_valid

,zho_Hans,kor_Hang
0,本方案的接触面膨胀型热熔断器涉及用于加热装置或加热部的接触面膨胀并且改善接触状态的热熔断器。,"본 고안의 접면확장 감온퓨즈는, 발열기구나 발열부위에 대한 접촉면을 확장시킴과 동시..."
1,可以沿着切割线高精度地切割要加工的物体。,절단 예정 라인에 따른 가공 대상물의 고정밀도 절단이 가능하게 된다.
2,"根据本发明,有效地清洁了铜(Cu)布线工艺中产生的受污染的晶片,从而使铜(Cu)污染导致的晶...","본 발명에 의하면, 구리(Cu) 배선 공정에서 발생하는 오염된 웨이퍼를 효율적으로 ..."
3,"本方案公开了使用无电极灯的照明灯具,该无电极灯通过高频感应线圈的感应电动势使磷光体发出由放电...",본 고안은 고주파 유도 코일의 유도 기전력에 의하여 방전관내의 플라즈마 발생에 의한...
4,"本发明涉及一种单向通信设备,其使得能够在发送终端与接收终端之间进行可靠的单向通信。","본 발명은 단방향 통신 장치로서, 송신 단말기와 수신 단말기 사이에 신뢰성 있는 단..."
...,...,...
14995,"本方案涉及一种电磁感应加热装置,更具体来说是涉及了识别用于炊具的容器,并控制该电磁感应加热装...","본 고안은 전자 유도가열장치로서, 더욱 상세하게는 조리기에 사용되는 용기를 식별하여..."
14996,"本方案涉及低压绕线管的扇形部分和模制壳体之间的组合结构,是为了在低压绕线管的扇形部分形成爪来...","본 고안은 저압보빈의 파이부와 몰딩케이스의 결합구조에 관한 것으로, 저압보빈의 파이..."
14997,"因此,由于无需诸如单独的焊接机或夹具的设备,仅在简单地插入组装工时的情况下进行旋钮的安装,因...",따라서 별도의 용착기나 지그 등의 장비없이 간단히 끼워주는 조립공수만으로 노브의 설...
14998,"根据本发明,具有无需再备用电源插座中安装另外的开关,而是利用无线开关来控制备用电源插座的优点。","본 발명에 따르면 대기전력콘센트에 별도로 스위치를 설치하지 않고, 무선스위치를 이용..."


find the max length of the encodings

In [9]:
from sklearn.model_selection import train_test_split

language_x = "zho_Hans"
language_y = "kor_Hang"
x_train = df_train[language_x].to_list()
y_train = df_train[language_y].to_list()
x_valid = df_valid[language_x].to_list()
y_valid = df_valid[language_y].to_list()

# 从训练集中抽出测试集，规模与验证集一致
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=len(x_valid), random_state=42)

In [10]:
# 保存到 CSV 文件
import pandas as pd

# 创建 DataFrame
train_df = pd.DataFrame({language_x: x_train, language_y: y_train})
valid_df = pd.DataFrame({language_x: x_valid, language_y: y_valid})
test_df = pd.DataFrame({language_x: x_test, language_y: y_test})

train_df.to_csv('data/train_data.csv', index=False)
valid_df.to_csv('data/valid_data.csv', index=False)
test_df.to_csv('data/test_data.csv', index=False)

Convert your encodings into torch Datasets object:

In [11]:
import numpy as np
from tqdm.notebook import tqdm

# 假设你已经有所有长度的列表
lengths = []

texts_all = x_train + y_train + x_valid + y_valid + x_test + y_test
for text in tqdm(texts_all):
    tokenized_text = tokenizer(text, truncation=False, padding=False)
    lengths.append(len(tokenized_text['input_ids']))

# 使用95%分位数来确定max_length
max_length = int(np.percentile(lengths, 95))
print(f"The 95% percentile length in the dataset is: {max_length}")

tokenizer.max_length = max_length

  0%|          | 0/270000 [00:00<?, ?it/s]

The 95% percentile length in the dataset is: 66


In [12]:
# Note: we're now creating separate encodings for the inputs and outputs.
# truncation: truncate the sequence to a shorter length, because sometimes a sequence may be too long for a model to handle
# padding: Padding is a strategy for ensuring tensors are rectangular by adding a special padding token to shorter sentences.
#     True or 'longest': Pad to the longest sequence in the batch (or no padding if only a single sequence if provided).
#     'max_length': Pad to a maximum length specified with the argument max_length or to the maximum acceptable input length for the model if that argument is not provided.
#     False or 'do_not_pad' (default): No padding (i.e., can output a batch with sequences of different lengths).
# return_tensors: If set 'pt', will return tensors instead of list of python integers. Acceptable values are PyTorch torch.Tensor objects.
# max_length (int, optional): Controls the maximu m length to use by one of the truncation/padding parameters.
tokenizer.src_lang = language_x
tokenizer.tgt_lang = language_y
print("tokenizing - source:{}, target:{}".format(language_x, language_y))
print("train encodings...")
train_encodings = tokenizer(x_train, text_target=y_train, truncation=True, padding="max_length", return_tensors="pt")
print("valid encodings...")
valid_encodings = tokenizer(x_valid, text_target=y_valid, truncation=True, padding="max_length", return_tensors="pt")
print("test encodings...")
test_encodings = tokenizer(x_test, text_target=y_test, truncation=True, padding="max_length", return_tensors="pt")

tokenizing - source:zho_Hans, target:kor_Hang
train encodings...
valid encodings...
test encodings...


In [13]:
import torch


class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, index):
        item = {key: val[index] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [14]:
train_dataset = TranslationDataset(train_encodings)
valid_dataset = TranslationDataset(valid_encodings)
test_dataset = TranslationDataset(test_encodings)

Load the pretrained model:

In [15]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_dir, device_map="auto")

Define your training arguments and train the model:

In [16]:
new_model_name = "zh2ko_based_on_{0}".format(model_name)

new_model_dir = os.path.join(data_disk_dir, new_model_name)

In [17]:
from transformers import Trainer, TrainingArguments, IntervalStrategy

# 每1/5 Epoch处验证、记录、保存一次
log_steps = len(x_train) / 5

# fp16：半精度运算，启用后提高一倍以上运算速度，不影响loss
# gradient_accumulation_steps：steps越大，速度越快，loss越高
# gradient_checkpointing：启用后，降低30%左右速度，节省显存2/3
# per_device_train_batch_size：size越大，GPU占用率越大，速度越快，loss越高，几乎成正比
training_args = TrainingArguments(new_model_dir,
                                  num_train_epochs=10,
                                  per_device_eval_batch_size=1,
                                  per_device_train_batch_size=1,
                                  gradient_accumulation_steps=1,
                                  gradient_checkpointing=False,
                                  fp16=True,
                                  warmup_ratio=0.1,
                                  eval_strategy=IntervalStrategy.STEPS,
                                  eval_steps=log_steps,
                                  logging_strategy=IntervalStrategy.STEPS,
                                  logging_steps=log_steps,
                                  save_strategy=IntervalStrategy.STEPS,
                                  save_steps=log_steps,
                                  save_total_limit=1,
                                  load_best_model_at_end=True
                                  )

In [18]:
from transformers import EarlyStoppingCallback


# ！！！猜测Transformer的原生Trainer存在显存泄露的问题，根据论坛上的神贴，做出如下修改：
# https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/13
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels


trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=valid_dataset,  # valid dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)

In [19]:
# 在测试数据集上进行预测
test_results = trainer.predict(test_dataset)

# 获取预测结果
predictions = test_results.predictions
labels = test_results.label_ids
metrics = test_results.metrics

In [20]:
# 输出评估指标
metrics

{'test_loss': 0.03576750308275223,
 'test_runtime': 653.417,
 'test_samples_per_second': 22.956,
 'test_steps_per_second': 22.956}

In [21]:
# 解码
references_array, target_texts_array = predictions
references = [tokenizer.decode(encoding, skip_special_tokens=True) for encoding in references_array]
target_texts = [tokenizer.decode(encoding, skip_special_tokens=True) for encoding in target_texts_array]

In [22]:
source_texts = x_test

df_result = pd.DataFrame({
    "source_texts": source_texts,
    "target_texts": target_texts,
    "references": references,
})
df_result

,source_texts,target_texts,references
0,提供了一种通过输出在移动通信终端上显示的附加内容作为广告数据来降低终端的价格并降低通信成本的方法。,이동통신 단말기에서 표시되는 부가 내용을 광고 데이터로 출력하도록 하여 단말기의 가...,이동통신 단말기에 표시되는 부가 컨 광고 데이터로 출력하여 함 단말기의 가격을 낮추...
1,"当在第九步中确定的环境亮度较亮时,将照原样输出在第四,第六和第七步中确定的声音值,并在第十一...","상기 제 9 단계에서 판별한 주변 밝기가 밝은 경우, 상기 제 4, 6, 7 단계에...","상기 제9단 결정정된 주 밝기가 밝 경우에는, 상기 제 4, 6, 7 단계에서 판된..."
2,有效地使用带摄像头的手机等中安装的发光二极管(LED)。,카메라 부착 휴대 전화기 등에 설치된 LED를 효율적으로 이용한다.,카메라가착 휴대폰 전화기 등에 설치된 발를 효율적으로 사용한다.
3,其涉及在将晶圆传送至工艺腔体的过程和对晶圆进行蚀刻工艺的过程中减少晶圆工艺面污染的半导体装置...,웨이퍼를 공정챔버로 이송시키는 과정과 웨이퍼에 식각 공정을 수행하는 과정에서 웨이퍼...,웨이퍼를 공정챔버로 이송하는 공정과 웨이퍼를 대한각공정을 진행하는 과정에서 웨이퍼 ...
4,"其提供有机发光器件,该有机发光器件能够将从有机发光单元发出的光聚集到一个地方以提高光利用效率...","유기 발광셀에서 방출되는 빛을 한곳으로 집중시켜 광 이용 효율을 향상시키고, 인접 ...",유기발광부로부터 발출된 광 한 곳으로에 집시켜 광 이용 효율을 높시키고 인 인접한소...
...,...,...,...
14995,根据本发明的防潮防湿配电箱的构造如下。,본 발명에 의한 냉각 방습 배전함은 다음과 같이 구성된다.,본 발명에 따른 방습수습 배전함 다음과 같이 구성된다.
14996,"本发明为了射频卡与射频终端机间的畅通通信,提出了射频终端机上附加的机械装置和为驱动该装置的电...",본 발명은 RF카드와 RF단말기간의 원활한 통신을 위하여 RF단말기에 추가될 기계적...,본 발명은 RF카드와 RF단말기간의 원활한 통신을 위해 RF단말기에 부되는 수장 장...
14997,本方案涉及通过发送和接收音响信号来控制设备的无电源远程控制设备。,본 고안은 음향신호의 송수신에 의해 기기를 제어하는 무전원 원격제어장치에 관한 것이다.,본 고안은 오향신호를 송수신을으로 의해 기기를 제어하는 무전원 원격제어장치에 관한 ...
14998,本发明涉及一种在呼叫中改变用于移动通信终端的FA以根据呼叫期间的系统负载实时地改变终端的FA...,본 발명은 통화 중에 시스템의 로드에 따라 실시간으로 단말의 FA를 변경하도록 하는...,본 발명은 통화 중 시스템 부드에 따라 단으로 단말의 FA를 변경하도록 통화 통화중...


In [23]:
file_name = "results_of_{0}".format(model_name)
df_result.to_excel("{0}.xlsx".format(file_name), index=False)
df_result.to_csv("{0}.csv".format(file_name), index=False)

In [24]:
from nltk.translate.bleu_score import corpus_bleu

# corpus_bleu 的结果更稳健，因为它是基于整个语料库的平均分数计算的，而 sentence_bleu 仅基于单个句子。
# 注：nltk3.8.1和python 3.12有点问题，要么降低python版本，要么按照以下链接的指导去修改bleu_score.py文件
# https://github.com/nltk/nltk/pull/3207
# https://github.com/nltk/nltk/blob/develop/nltk/translate/bleu_score.py
score = corpus_bleu([[reference] for reference in references], target_texts)
score

0.7068739162535101

In [27]:
# trainer.train(resume_from_checkpoint=True)
trainer.train(resume_from_checkpoint="autodl-tmp/zh2ko_based_on_nllb-200-distilled-600M/checkpoint-315000")
# trainer.train(resume_from_checkpoint=False)

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Step,Training Loss,Validation Loss
336000,0.031100,0.035885
357000,0.032000,0.035748


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length':

Save your fine-tuned model and tokenizer:

In [28]:
trainer.save_model(new_model_dir)
tokenizer.save_pretrained(new_model_dir)
trainer.save_state()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


In [29]:
# 在测试数据集上进行预测
test_results = trainer.predict(test_dataset)

# 获取预测结果
predictions = test_results.predictions
labels = test_results.label_ids
metrics = test_results.metrics

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [30]:
# 输出评估指标
metrics

{'test_loss': 0.03576750308275223,
 'test_runtime': 655.1329,
 'test_samples_per_second': 22.896,
 'test_steps_per_second': 22.896}

In [31]:
references_array, target_texts_array = predictions

# 对两个数组中的每一个编码进行解码
references = [tokenizer.decode(encoding, skip_special_tokens=True) for encoding in references_array]
target_texts = [tokenizer.decode(encoding, skip_special_tokens=True) for encoding in target_texts_array]

In [32]:
source_texts = x_test

df_result = pd.DataFrame({
    "source_texts": source_texts,
    "target_texts": target_texts,
    "references": references,
})
df_result

,source_texts,target_texts,references
0,提供了一种通过输出在移动通信终端上显示的附加内容作为广告数据来降低终端的价格并降低通信成本的方法。,이동통신 단말기에서 표시되는 부가 내용을 광고 데이터로 출력하도록 하여 단말기의 가...,이동통신 단말기에 표시되는 부가 컨 광고 데이터로 출력하여 함 단말기의 가격을 낮추...
1,"当在第九步中确定的环境亮度较亮时,将照原样输出在第四,第六和第七步中确定的声音值,并在第十一...","상기 제 9 단계에서 판별한 주변 밝기가 밝은 경우, 상기 제 4, 6, 7 단계에...","상기 제9단 결정정된 주 밝기가 밝 경우에는, 상기 제 4, 6, 7 단계에서 판된..."
2,有效地使用带摄像头的手机等中安装的发光二极管(LED)。,카메라 부착 휴대 전화기 등에 설치된 LED를 효율적으로 이용한다.,카메라가착 휴대폰 전화기 등에 설치된 발를 효율적으로 사용한다.
3,其涉及在将晶圆传送至工艺腔体的过程和对晶圆进行蚀刻工艺的过程中减少晶圆工艺面污染的半导体装置...,웨이퍼를 공정챔버로 이송시키는 과정과 웨이퍼에 식각 공정을 수행하는 과정에서 웨이퍼...,웨이퍼를 공정챔버로 이송하는 공정과 웨이퍼를 대한각공정을 진행하는 과정에서 웨이퍼 ...
4,"其提供有机发光器件,该有机发光器件能够将从有机发光单元发出的光聚集到一个地方以提高光利用效率...","유기 발광셀에서 방출되는 빛을 한곳으로 집중시켜 광 이용 효율을 향상시키고, 인접 ...",유기발광부로부터 발출된 광 한 곳으로에 집시켜 광 이용 효율을 높시키고 인 인접한소...
...,...,...,...
14995,根据本发明的防潮防湿配电箱的构造如下。,본 발명에 의한 냉각 방습 배전함은 다음과 같이 구성된다.,본 발명에 따른 방습수습 배전함 다음과 같이 구성된다.
14996,"本发明为了射频卡与射频终端机间的畅通通信,提出了射频终端机上附加的机械装置和为驱动该装置的电...",본 발명은 RF카드와 RF단말기간의 원활한 통신을 위하여 RF단말기에 추가될 기계적...,본 발명은 RF카드와 RF단말기간의 원활한 통신을 위해 RF단말기에 부되는 수장 장...
14997,本方案涉及通过发送和接收音响信号来控制设备的无电源远程控制设备。,본 고안은 음향신호의 송수신에 의해 기기를 제어하는 무전원 원격제어장치에 관한 것이다.,본 고안은 오향신호를 송수신을으로 의해 기기를 제어하는 무전원 원격제어장치에 관한 ...
14998,本发明涉及一种在呼叫中改变用于移动通信终端的FA以根据呼叫期间的系统负载实时地改变终端的FA...,본 발명은 통화 중에 시스템의 로드에 따라 실시간으로 단말의 FA를 변경하도록 하는...,본 발명은 통화 중 시스템 부드에 따라 단으로 단말의 FA를 변경하도록 통화 통화중...


In [33]:
file_name = "results_of_{0}".format(new_model_name)
df_result.to_excel("{0}.xlsx".format(file_name), index=False)
df_result.to_csv("{0}.csv".format(file_name), index=False)

In [34]:
from nltk.translate.bleu_score import corpus_bleu

# corpus_bleu 的结果更稳健，因为它是基于整个语料库的平均分数计算的，而 sentence_bleu 仅基于单个句子。
# 注：nltk3.8.1和python 3.12有点问题，要么降低python版本，要么按照以下链接的指导去修改bleu_score.py文件
# https://github.com/nltk/nltk/pull/3207
# https://github.com/nltk/nltk/blob/develop/nltk/translate/bleu_score.py
score = corpus_bleu([[reference] for reference in references], target_texts)
score

0.7068739162535101